# Analysing density maps

Make sure that that essential packages are installed  

In [ ]:
! pip install ovito
! pip install h5py
! pip install natsort

Create an output folder

In [ ]:
! mkdir output

Run the command line tool on the trajectory file and output folder. Save to hdf5.

In [ ]:
! python densityField.py HISTORY.000 output 

In [ ]:
! ls output

You can read the output easily with `h5py`

In [ ]:
import h5py

h5 = h5py.File("output/hist-data-dl1.0.h5")
h5.keys()


A single frame is a 3d array

In [ ]:
h5["frame_0"].shape


## Options

### Changing binsize

To change binsize, use the `--dl` option. Its default value is `1`

In [ ]:
! python densityField.py HISTORY.000 output --dl 0.5

### Selecting particle type

To select the particle type to bin, use the `--selection` option.

In [ ]:
! mkdir outputOW
! python densityField.py HISTORY.000 outputOW --dl 2.5 --selection OW

### Plotting 2d snapshots 

Average maps at every time can be stored with the `--map2d` option, indicating the axis for the averaging `0,1,2`

In [ ]:
! python densityField.py HISTORY.000 outputOW --dl 0.5 --selection OW --map2d 0

## Statistics

### Time average

We can take the time average and variance of the trajectory by reading all sorted frames and by using `numpy`

In [ ]:
import numpy as np
from natsort import realsorted
import h5py

h5 = h5py.File("outputOW/hist-data-dl2.5.h5")
ns = np.array([h5[k] for k in realsorted(h5.keys()) if "frame" in k])
# averaging along time
avg_n = np.mean(ns, axis=0)
var_n = np.var(ns, axis=0)


### Average along a spatial dimension

The resulting arrays are averaged using `numpy`

For example, the average along the x dimension is done 

In [ ]:
n_yz = avg_n.mean(axis=0)


## Plotting

## 2D plots

Two-dimensional plots can be produced by `matplotlib`

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.matshow(n_yz)
plt.colorbar()


## 3D plots

A convenient 3D plotter for surfaces is `pyvista`

In [ ]:
pip install 'pyvista[all]' jupyterlab

In [ ]:
import pyvista as pv

# create image canvas
mesh = pv.ImageData()
# create grid for cell data
mesh.dimensions = np.array(avg_n.shape)
# assign numpy array to point data
mesh.point_data["values"] = avg_n.flatten(order="F")
# plot orthogonal projections
slices = mesh.slice_orthogonal()
dargs = dict(cmap="gist_ncar_r")

p = pv.Plotter()
p.add_mesh(slices, **dargs)
p.show()


Contours are also possible.

In [ ]:
mesh.contour([0.6]).plot()